In [1]:
import transformed_data as data
import dataframeinfo as info
import pandas as pd
import numpy as np
import missingno as msno

In [2]:
# Assign data from transformed_data to variable "loans". 
loans = data.loans
loans.head()

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term_months,int_rate,instalment,grade,sub_grade,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
0,38676116,41461848,8000,8000.0,8000.0,36.0,7.49,248.82,A,A4,...,0.0,0.0,2022-01,248.82,2022-02,2022-01,0.0,150 days 00:00:00,1,INDIVIDUAL
1,38656203,41440010,13200,13200.0,13200.0,36.0,6.99,407.52,A,A3,...,0.0,0.0,2022-01,407.52,2022-02,2022-01,0.0,NaT,1,INDIVIDUAL
2,38656154,41439961,16000,16000.0,16000.0,36.0,7.49,497.63,A,A4,...,0.0,0.0,2021-10,12850.16,NaN,2021-10,0.0,NaT,1,INDIVIDUAL
3,38656128,41439934,15000,15000.0,15000.0,36.0,14.31,514.93,C,C4,...,0.0,0.0,2021-06,13899.67,NaN,2021-06,0.0,NaT,1,INDIVIDUAL
4,38656121,41439927,15000,15000.0,15000.0,36.0,6.03,456.54,A,A1,...,0.0,0.0,2022-01,456.54,2022-02,2022-01,0.0,NaT,1,INDIVIDUAL


In [3]:
# Create instance of the DataFrameInfo class.
loans_info = info.DataFrameInfo(loans)

In [4]:
loans_info.percent_na()

Column               Missing values (%) 
 funded_amount                   5.54
term_months                     8.80
int_rate                        9.53
employment_years                3.91
mths_since_last_delinq         57.17
mths_since_last_record         88.60
last_payment_date               0.13
next_payment_date              60.13
last_credit_pull_date           0.01
collections_12_mths_ex_med      0.09
mths_since_last_major_derog    86.17
dtype: float64
